In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
import os
import glob
from collections import Counter

In [2]:
annotations = json.load(open(os.path.join('trainval','annotations','bbox-annotations.json'),'r'))
w = []
h = []
for img in annotations['images']:
    w.append(img['width'])
    h.append(img['height'])
w = np.array(w)
h = np.array(h)

In [4]:
lo_w = np.median(w)-3*np.std(w)
hi_w = np.median(w)+3*np.std(w)

lo_h = np.median(h)-3*np.std(h)
hi_h = np.median(h)+3*np.std(h)

In [5]:
outlier_image_ids = []
outlier_image_names = []
for img in annotations['images']:
    w = img['width']
    h = img['height']
    if not ((lo_w<=w<=hi_w) and (lo_h<=h<=hi_h)):
        outlier_image_ids.append(img['id'])
        outlier_image_names.append(img['file_name'])
print(len(outlier_image_ids), outlier_image_ids)
print(outlier_image_names)

18 [28, 86, 364, 721, 733, 863, 896, 930, 939, 1241, 1424, 1557, 1667, 1692, 1758, 1996, 2110, 2129]
['image_000000029.jpg', 'image_000000087.jpg', 'image_000000365.jpg', 'image_000000722.jpg', 'image_000000734.jpg', 'image_000000864.jpg', 'image_000000897.jpg', 'image_000000931.jpg', 'image_000000940.jpg', 'image_000001242.jpg', 'image_000001425.jpg', 'image_000001558.jpg', 'image_000001668.jpg', 'image_000001693.jpg', 'image_000001759.jpg', 'image_000001997.jpg', 'image_000002111.jpg', 'image_000002130.jpg']


In [7]:
# Remove these images from the training set.
for img in outlier_image_names:
    img_path = os.path.join('trainval','images',img)
    os.remove(img_path)

In [8]:
outlier_image_ids = set(outlier_image_ids)

for i, img in enumerate(annotations['images']):
    if img['id'] in outlier_image_ids:
        annotations['images'].pop(i)

for i, annot in enumerate(annotations['annotations']):
    if annot['image_id'] in outlier_image_ids:
        annotations['annotations'].pop(i)

In [9]:
json.dump(annotations, open(os.path.join('trainval','annotations','annotations-cleaned.json'),'w'))